In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from pmdarima.arima import auto_arima
import pandas as pd


In [2]:
df = pd.read_csv('srednjeni_podaci.csv', parse_dates=True)

In [3]:
df.head


<bound method NDFrame.head of      Unnamed: 0  godina       jan       feb        mar        apr        maj  \
0             0  1887.0  0.295349  2.032558   6.925581  12.193798  17.117054   
1             1  1888.0 -6.500000 -2.300000   7.100000  10.900000  15.900000   
2             2  1889.0 -4.000000 -0.900000   3.700000  11.000000  18.200000   
3             3  1890.0  1.300000 -1.700000   6.500000  13.000000  17.600000   
4             4  1891.0 -6.400000 -4.800000   6.500000   9.400000  19.100000   
..          ...     ...       ...       ...        ...        ...        ...   
126         126  2013.0  3.200000  4.400000   6.600000  14.800000  18.900000   
127         127  2014.0  5.200000  7.600000  10.800000  13.600000  16.900000   
128         128  2015.0  3.800000  4.000000   8.100000  13.200000  18.800000   
129         129  2016.0  2.400000  9.000000   9.000000  15.400000  17.300000   
130         130  2017.0  0.295349  2.032558   6.925581  12.193798  17.117054   

         

In [4]:
df = df.drop(df.columns[0], axis=1)

In [5]:
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=False)

In [6]:

# Definisanje i treniranje modela
model = auto_arima(train_data['prosek'], trace=True, error_action='ignore', suppress_warnings=True)
model.fit(train_data['prosek'])

# Predviđanja za testni skup
forecast = model.predict(n_periods=len(test_data))
forecast = pd.DataFrame(forecast, index=test_data.index, columns=['Prediction'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.52 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=293.123, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=266.133, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.19 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=291.134, Time=0.02 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=262.660, Time=0.05 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=257.182, Time=0.06 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=253.075, Time=0.10 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=254.972, Time=0.12 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.52 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.42 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.59 sec
 ARIMA(4,1,0)(0,0,0)[0]             : AIC=251.424, Time=0.07 sec
 ARIMA(3,1,0)(0,0,0)[0]             : AIC=255.417, Time=0.06 sec
 ARIMA(5,1,0)(0,0,0)[0]             : AIC=253.334, Time=0.08 sec
 A

In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Izračunavanje MSE
mse = mean_squared_error(test_data['prosek'], forecast['Prediction'])

# Izračunavanje MAE
mae = mean_absolute_error(test_data['prosek'], forecast['Prediction'])

# Izračunavanje RMSE
rmse = np.sqrt(mse)

# Izračunavanje MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(test_data['prosek'], forecast['Prediction'])

# Ispisivanje rezultata
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Percentage Error (MAPE):", mape)

Mean Squared Error (MSE): 1.6036087998820945
Mean Absolute Error (MAE): 1.0704202729101386
Root Mean Squared Error (RMSE): 1.2663367640095167
Mean Absolute Percentage Error (MAPE): 7.99357247162468


In [8]:
# Definisanje parametara pretrage
stepwise_model = auto_arima(train_data['prosek'], 
                            start_p=1, start_q=1,
                            max_p=3, max_q=3, 
                            m=12,  # broj vremenskih koraka u jednoj sezoni
                            start_P=0, seasonal=True,
                            d=1, D=1, trace=True,
                            error_action='ignore',  
                            suppress_warnings=True, 
                            stepwise=True)

# Treniranje modela
stepwise_model.fit(train_data['prosek'])

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=inf, Time=1.26 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=334.866, Time=0.04 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=270.679, Time=0.29 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=1.64 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=303.715, Time=0.05 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=256.712, Time=0.63 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=254.002, Time=1.85 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=252.007, Time=0.45 sec
 ARIMA(1,1,0)(0,1,1)[12]             : AIC=inf, Time=0.72 sec
 ARIMA(1,1,0)(1,1,2)[12]             : AIC=254.006, Time=2.63 sec
 ARIMA(1,1,0)(0,1,2)[12]             : AIC=252.085, Time=4.14 sec
 ARIMA(1,1,0)(2,1,2)[12]             : AIC=255.360, Time=1.11 sec
 ARIMA(0,1,0)(1,1,1)[12]             : AIC=282.878, Time=0.19 sec
 ARIMA(2,1,0)(1,1,1)[12]             : AIC=251.730, Time=0.37 sec
 ARIMA(2,1,0)(0,1,1)[12]             : AIC=in

ARIMA(order=(3, 1, 1), scoring_args={}, seasonal_order=(1, 1, 1, 12),
      suppress_warnings=True, with_intercept=False)

In [9]:
# Predviđanja za testni skup
forecast_stepwise = stepwise_model.predict(n_periods=len(test_data))
forecast_stepwise = pd.DataFrame(forecast_stepwise, index=test_data.index, columns=['Prediction'])

In [10]:
# Izračunavanje metrika
mse = mean_squared_error(test_data['prosek'], forecast_stepwise)
mae = mean_absolute_error(test_data['prosek'], forecast_stepwise)
rmse = np.sqrt(mse)

# Ispis rezultata
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)


Mean Squared Error (MSE): 1.3416653573237585
Mean Absolute Error (MAE): 0.9727812672897737
Root Mean Squared Error (RMSE): 1.1583027917275166


In [11]:

stepwise_model_leto = auto_arima(train_data['leto'], 
                            start_p=1, start_q=1,
                            max_p=3, max_q=3, 
                            m=4,  # broj vremenskih koraka u jednoj sezoni
                            start_P=0, seasonal=True,
                            d=1, D=1, trace=True,
                            error_action='ignore',  
                            suppress_warnings=True, 
                            stepwise=True)

stepwise_model_leto.fit(train_data['leto'])

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[4]             : AIC=inf, Time=0.28 sec
 ARIMA(0,1,0)(0,1,0)[4]             : AIC=378.572, Time=0.02 sec
 ARIMA(1,1,0)(1,1,0)[4]             : AIC=360.040, Time=0.03 sec
 ARIMA(0,1,1)(0,1,1)[4]             : AIC=inf, Time=0.14 sec
 ARIMA(1,1,0)(0,1,0)[4]             : AIC=377.550, Time=0.03 sec
 ARIMA(1,1,0)(2,1,0)[4]             : AIC=359.640, Time=0.06 sec
 ARIMA(1,1,0)(2,1,1)[4]             : AIC=inf, Time=0.42 sec
 ARIMA(1,1,0)(1,1,1)[4]             : AIC=inf, Time=0.23 sec
 ARIMA(0,1,0)(2,1,0)[4]             : AIC=362.405, Time=0.03 sec
 ARIMA(2,1,0)(2,1,0)[4]             : AIC=352.246, Time=0.08 sec
 ARIMA(2,1,0)(1,1,0)[4]             : AIC=353.324, Time=0.05 sec
 ARIMA(2,1,0)(2,1,1)[4]             : AIC=inf, Time=1.37 sec
 ARIMA(2,1,0)(1,1,1)[4]             : AIC=inf, Time=0.90 sec
 ARIMA(3,1,0)(2,1,0)[4]             : AIC=341.734, Time=0.09 sec
 ARIMA(3,1,0)(1,1,0)[4]             : AIC=343.413, Time=0.06 sec
 ARIMA

ARIMA(order=(3, 1, 0), scoring_args={}, seasonal_order=(2, 1, 0, 4),
      suppress_warnings=True, with_intercept=False)

In [12]:

forecast_stepwise = stepwise_model_leto.predict(n_periods=len(test_data))
forecast_stepwise = pd.DataFrame(forecast_stepwise, index=test_data.index, columns=['Prediction'])

In [13]:

mse = mean_squared_error(test_data['leto'], forecast_stepwise)
mae = mean_absolute_error(test_data['leto'], forecast_stepwise)
rmse = np.sqrt(mse)


print("Mean Squared Error (MSE)- Summer:", mse)
print("Mean Absolute Error (MAE)- Summer:", mae)
print("Root Mean Squared Error (RMSE)- Summer:", rmse)


Mean Squared Error (MSE)- Summer: 2.3985362793720935
Mean Absolute Error (MAE)- Summer: 1.29960859621733
Root Mean Squared Error (RMSE)- Summer: 1.5487208526303549


In [14]:

stepwise_model_zima = auto_arima(train_data['zima'], 
                            start_p=1, start_q=1,
                            max_p=3, max_q=3, 
                            m=4, 
                            start_P=0, seasonal=True,
                            d=1, D=1, trace=True,
                            error_action='ignore',  
                            suppress_warnings=True, 
                            stepwise=True)

# Treniranje modela
stepwise_model_zima.fit(train_data['zima'])

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[4]             : AIC=inf, Time=2.10 sec
 ARIMA(0,1,0)(0,1,0)[4]             : AIC=582.452, Time=0.03 sec
 ARIMA(1,1,0)(1,1,0)[4]             : AIC=510.016, Time=0.08 sec
 ARIMA(0,1,1)(0,1,1)[4]             : AIC=inf, Time=0.27 sec
 ARIMA(1,1,0)(0,1,0)[4]             : AIC=544.750, Time=0.06 sec
 ARIMA(1,1,0)(2,1,0)[4]             : AIC=498.867, Time=0.08 sec
 ARIMA(1,1,0)(2,1,1)[4]             : AIC=inf, Time=0.41 sec
 ARIMA(1,1,0)(1,1,1)[4]             : AIC=inf, Time=0.19 sec
 ARIMA(0,1,0)(2,1,0)[4]             : AIC=536.579, Time=0.07 sec
 ARIMA(2,1,0)(2,1,0)[4]             : AIC=495.719, Time=0.07 sec
 ARIMA(2,1,0)(1,1,0)[4]             : AIC=506.367, Time=0.05 sec
 ARIMA(2,1,0)(2,1,1)[4]             : AIC=inf, Time=1.02 sec
 ARIMA(2,1,0)(1,1,1)[4]             : AIC=inf, Time=0.34 sec
 ARIMA(3,1,0)(2,1,0)[4]             : AIC=493.723, Time=0.10 sec
 ARIMA(3,1,0)(1,1,0)[4]             : AIC=507.078, Time=0.07 sec
 ARIMA

ARIMA(order=(3, 1, 0), scoring_args={}, seasonal_order=(2, 1, 0, 4),
      suppress_warnings=True, with_intercept=False)

In [15]:

forecast_stepwise = stepwise_model_zima.predict(n_periods=len(test_data))
forecast_stepwise = pd.DataFrame(forecast_stepwise, index=test_data.index, columns=['Prediction'])

In [16]:

mse = mean_squared_error(test_data['zima'], forecast_stepwise)
mae = mean_absolute_error(test_data['zima'], forecast_stepwise)
rmse = np.sqrt(mse)

# Ispis rezultata
print("Mean Squared Error (MSE) - winter:", mse)
print("Mean Absolute Error (MAE)- winter:", mae)
print("Root Mean Squared Error (RMSE)- winter:", rmse)

Mean Squared Error (MSE) - winter: 10.702812036344689
Mean Absolute Error (MAE)- winter: 2.7980409453931108
Root Mean Squared Error (RMSE)- winter: 3.271515250819517
